In [104]:
from astroquery.mast import Catalogs
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm

In [105]:
# Load catalogue of blazars
blazars = pd.read_csv('E:\\catalogues\\blazar_cat-3.csv')

# Initialize empty lists of desired parameters
ra_list, dec_list, GAIAmag_list, GAIA_list = [], [], [], []

for index, row in tqdm(blazars.iterrows()):
    
    # Format coordinates
    coords = str(blazars['RA'][index]) + ' ' + str(blazars['DEC'][index])
    bright = blazars['Gaia_G_Magnitude'][index]

    # Query the TIC within approximately ten TESS pixels
    catalog_data = Catalogs.query_object(coords, radius = 0.055, catalog="TIC").to_pandas()
    
    # Sort by difference between blazar brightness and star brightness
    difs = []
    for index, row in catalog_data.iterrows():
        difs.append(abs(bright - row['GAIAmag']))
    catalog_data['mag_difs'] = difs
    catalog_data = catalog_data.sort_values('mag_difs',ignore_index=True)
    
    # Select the second object in the dataframe (to exclude the blazar itself), unless that object isn't a star
    selected_object = 1
    while catalog_data['objType'][selected_object] != 'STAR':
        selected_object = selected_object + 1 
    
    # Append selected object to empty lists
    ra_list.append(catalog_data['ra'][selected_object])
    dec_list.append(catalog_data['dec'][selected_object])
    GAIAmag_list.append(catalog_data['GAIAmag'][selected_object])
    GAIA_list.append(catalog_data['GAIA'][selected_object])
    
# Append lists of objects to blazar catalogue
blazars['match_RA'] = ra_list
blazars['match_DEC'] = dec_list
blazars['match_ID'] = GAIA_list
blazars['match_MAG'] = GAIAmag_list

708it [14:15,  1.21s/it]


In [107]:
blazars.to_csv('E:\\catalogues\\blazar_cat-4.csv')